In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # this loads the .env file automatically

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"]  = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching for potential venues in San Francisco that can accommodate a Tech Innovation Conference. I'll use the "Search the internet" tool to find some options.

Action: Search the internet

Action Input: ```{"search_query": "Tech Innovation Conference venues in San Francisco"}```Thought: It seems there was an issue with the format of my input. I need to correct the format and try again.

Action: Search the internet

Action Input: ```{"search_query": "Tech Innovation Conference venues in San Francisco"}```Thought: It seems there's an issue with the input format. I need to ensure the input is correctly formatted as a dictionary with proper quotes. Let me correct this and try again.

Action: Search the internet

Action Input: ```{"search_query": "Tech Innovation Co

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '255 S Airport Blvd, South San Francisco, CA 94080',
 'booking_status': 'For more information, contact Kathy at 650-877-5205.',
 'capacity': 20500,
 'name': 'South San Francisco Conference Center'}


In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

```markdown
# Tech Innovation Conference Marketing Report

## Objective
Promote the Tech Innovation Conference to engage at least 500 potential attendees.

## Target Audience
Based on the gathered information, the target audience for the Tech Innovation Conference includes:

1. **Tech Professionals**: Individuals working in various roles within the technology sector, including software developers, engineers, data scientists, and IT specialists.
2. **Industry Professionals**: Executives, managers, and decision-makers from tech companies, startups, and established enterprises.
3. **Entrepreneurs and Startups**: Founders and key personnel from startups looking to innovate, network, and seek investment opportunities.
4. **Investors**: Venture capitalists, angel investors, and other financial backers interested in emerging technologies and startups.
5. **Students and Academia**: Students and researchers from universities and research institutions focusing on technology and innovation.
6. **Tech Enthusiasts**: Individuals passionate about technology and eager to learn about the latest trends and innovations.

## Compelling Content Ideas
1. **Keynote Speeches**: Invite renowned industry leaders, innovators, and thought leaders to deliver keynote speeches on cutting-edge topics such as AI, blockchain, IoT, and cybersecurity.
2. **Panel Discussions**: Organize panel discussions featuring experts from various tech domains to discuss relevant topics like digital transformation, emerging technologies, and the future of tech innovation.
3. **Workshops and Training Sessions**: Offer hands-on workshops and training sessions on topics like data science, machine learning, coding, and cybersecurity to provide practical skills and knowledge.
4. **Demo and Pitch Sessions**: Provide a platform for startups and tech companies to showcase their products, services, and innovations through demo sessions and pitch competitions.
5. **Networking Events**: Facilitate networking opportunities through mixer events, speed networking sessions, and dedicated networking areas to foster connections among attendees.
6. **Exhibitor Hall**: Set up an exhibitor hall where tech companies, startups, and service providers can display their products, services, and innovations, and engage with attendees.
7. **Inspirational Talks**: Invite successful entrepreneurs and tech leaders to share their journeys, challenges, and lessons learned to inspire and motivate the audience.
8. **Interactive Sessions**: Include interactive sessions such as hackathons, coding challenges, and Q&A sessions with experts to engage the audience and encourage participation.
9. **Focus on Emerging Technologies**: Dedicate sessions to emerging technologies like AI, blockchain, IoT, and quantum computing to highlight their potential and applications.
10. **Industry-specific Tracks**: Create tracks or sessions focused on specific industries such as healthcare, finance, education, and manufacturing to showcase tech innovations tailored to those sectors.

## Marketing Strategies
1. **Social Media Campaigns**: Utilize platforms like LinkedIn, Twitter, and Facebook to reach a broader audience. Share engaging content, behind-the-scenes looks, and live updates during the conference.
2. **Email Campaigns**: Send targeted email invitations to potential attendees, highlighting key speakers, sessions, and networking opportunities.
3. **Partnerships and Collaborations**: Collaborate with tech influencers, industry leaders, and relevant organizations to expand reach and credibility.
4. **Press Releases**: Issue press releases to local and tech media outlets to generate buzz and attract media coverage.
5. **Event Website and Registration**: Ensure the event website is user-friendly and the registration process is seamless. Highlight key features, benefits, and testimonials from past attendees.
6. **Paid Advertising**: Consider using paid advertising on social media and search engines to reach a larger audience and drive registrations.

## Engagement Activities
1. **Pre-Event Polls and Surveys**: Engage potential attendees before the event by conducting polls and surveys to understand their interests and preferences.
2. **Live Social Media Engagement**: Encourage attendees to share their experiences using a dedicated hashtag and offer prizes for the most engaging posts.
3. **Interactive Exhibitor Booths**: Create interactive and engaging exhibitor booths that offer hands-on demos, giveaways, and interactive activities.
4. **Post-Event Feedback and Surveys**: Collect feedback from attendees after the event to understand their experience and identify areas for improvement.

By implementing these strategies and focusing on the target audience and compelling content, the Tech Innovation Conference can achieve its goal of engaging at least 500 potential attendees and creating a valuable and memorable experience for all participants.
```